In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings



c:\Users\Administrator\.conda\envs\cv_parser\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('skills.csv')
df.head()

,S.No.,S.No..1,Name,Assessment,AI,Mock,Description,Categories,Skills/Technologies,type,stype,301
0,1.0,1.0,Java,True,True,True,Mock interview designed to assess your Java pr...,"Basic Syntax and Language Fundamentals, Object...","1, 42, 53, 113, 118, 160",1.0,1.0,NaN
1,2.0,2.0,MySQL,True,True,True,Mock interview tailored to evaluate your MySQL...,"Data Definition Language, Data Manipulation La...",NaN,1.0,1.0,NaN
2,3.0,3.0,MongoDB,True,True,True,Mock interview designed to assess your MongoDB...,"Introduction to NoSQL, MongoDB Basics, CRUD Op...",NaN,1.0,1.0,NaN
3,4.0,4.0,NodeJS,True,True,True,Mock interview tailored to evaluate your Node....,"JavaScript Fundamentals, Object-Oriented JavaS...",NaN,1.0,1.0,NaN
4,5.0,5.0,Python,True,True,True,Mock interview designed to assess your Python ...,"Basics, Control Structures, Functions, Data St...",NaN,1.0,1.0,NaN


In [3]:
df_cleaned = pd.read_csv('csv_db.csv')

df_cleaned['s_no'] = df_cleaned['s_no'].astype(int)

df_cleaned.head()

,s_no,name
0,1,Java
1,2,MySQL
2,3,MongoDB
3,4,NodeJS
4,5,Python


In [4]:
def normalize_skill(skill: str) -> str:
    return (
        skill.strip()
         .lower()
         .replace(" ", "")
    )


In [4]:
def normalize(cllgge: str) -> str:
    return (
    cllgge.strip()
        .lower()
        .replace("-", " ")
        .replace(".", "")
        .replace(",", "")
        .replace("/", " ")
        .replace("  ", " ")
)


In [6]:
model = SentenceTransformer('all-mpnet-base-v2')
df_cleaned['name'] = df_cleaned['name'].astype(str).apply(normalize_skill)
skill_texts = df_cleaned['name'].tolist()
skill_embeddings = model.encode(skill_texts, normalize_embeddings=True)


In [6]:
model = SentenceTransformer('all-mpnet-base-v2')

In [15]:

chroma_client = chromadb.PersistentClient(path="chroma_db_api")
collection = chroma_client.get_or_create_collection(
    name="skills",
    metadata={"hnsw:space": "cosine"}       
)

'''collection.add(
    ids=[str(i) for i in df_cleaned['s_no']], 
    embeddings=skill_embeddings.tolist(),
    metadatas=[{"skill": s} for s in skill_texts],
    documents=skill_texts                 
)'''

'collection.add(\n    ids=[str(i) for i in df_cleaned[\'s_no\']], \n    embeddings=skill_embeddings.tolist(),\n    metadatas=[{"skill": s} for s in skill_texts],\n    documents=skill_texts                 \n)'

In [16]:
def lookup_skill_id(raw_input: str, k: int = 1):
    
    q_emb = model.encode([raw_input], normalize_embeddings=True)[0]

    results = collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=k,
        include=['metadatas', 'distances']  # 'ids' always returned
    )

    ids = results['ids'][0] if results['ids'] else []
    scores = results['distances'][0] if results['distances'] else []
    meta = results['metadatas'][0] if results['metadatas'] else []

    matches = []
    for i in range(len(ids)):
        matches.append({
            "skill_id": ids[i],
            "skill": meta[i].get("skill", ""),
            "score": scores[i]
        })

    return matches

# Example Usage
print(lookup_skill_id("spray")) 
print(lookup_skill_id("counter strafe", k=3)) 
print(lookup_skill_id("machine learning", k=3))  
print(lookup_skill_id("deep learning", k=3))  
print(lookup_skill_id("artificial intelligence", k=3)) 
print(lookup_skill_id("cloud computing", k=3))  
print(lookup_skill_id("web development", k=3))
print(lookup_skill_id("data science", k=3))
print(lookup_skill_id("software engineering", k=3))
print(lookup_skill_id("project management", k=3))
print(lookup_skill_id("cybersecurity", k=3))
print(lookup_skill_id("database management", k=3))
print(lookup_skill_id("networking", k=3))
print(lookup_skill_id("mobile app development", k=3))
print(lookup_skill_id("game development", k=3))
print(lookup_skill_id("user experience design", k=3))
print(lookup_skill_id("data visualization", k=3))
print(lookup_skill_id("business analysis", k=3))


[{'skill_id': '10000', 'skill': 'spraycontrol', 'score': 0.3199576735496521}]
[{'skill_id': '9999', 'skill': 'counterstrafing', 'score': 0.38906341791152954}, {'skill_id': '90', 'skill': 'dart', 'score': 0.6202440857887268}, {'skill_id': '237', 'skill': 'tally', 'score': 0.6452953815460205}]
[{'skill_id': '254', 'skill': 'machinelearning', 'score': 0.24503594636917114}, {'skill_id': '273', 'skill': 'ai/ml', 'score': 0.3025161027908325}, {'skill_id': '217', 'skill': 'deeplearning', 'score': 0.3885498642921448}]
[{'skill_id': '217', 'skill': 'deeplearning', 'score': 0.21855789422988892}, {'skill_id': '254', 'skill': 'machinelearning', 'score': 0.33484703302383423}, {'skill_id': '273', 'skill': 'ai/ml', 'score': 0.35733991861343384}]
[{'skill_id': '264', 'skill': 'artificialintelligence', 'score': 0.1951366662979126}, {'skill_id': '273', 'skill': 'ai/ml', 'score': 0.2528197765350342}, {'skill_id': '254', 'skill': 'machinelearning', 'score': 0.38974201679229736}]
[{'skill_id': '123', 'skil

TS: not typescript;
Java Script: not javascript but scripting;
Edge case:  CPP Cplusplus

def normalize_skill(skill: str) -> str:
    return (
        skill.strip()
         .lower()
    )

In [10]:
def normalize_skill(skill: str) -> str:
    return (
        skill.strip()
         .lower()
         .replace(" ", "")
    )

print(normalize_skill("  Data Structures  "))  # Example usage of normalization function
print(normalize_skill("Machine Learning"))  # Another example usage 

datastructures
machinelearning


In [11]:
df_clg = pd.read_csv('india_engg_colleges_updated(in).csv')
df_clg.head()

,CollegeID,College Name,University,City,State
0,1,National Institute of Technology Rourkela,NaN,Rourkela,Odisha
1,2,Visvesvaraya National Institute of Technology ...,NaN,Nagpur,Maharashtra
2,3,Netaji Subhas University of Technology,NaN,New Delhi,Delhi
3,4,Birla Institute of Technology,NaN,Ranchi,Jharkhand
4,5,International Institute of Information Technology,NaN,Hyderabad,Telangana


In [12]:
df_clg = df_clg[['CollegeID', 'College Name']]

In [13]:
df_clg.head()

,CollegeID,College Name
0,1,National Institute of Technology Rourkela
1,2,Visvesvaraya National Institute of Technology ...
2,3,Netaji Subhas University of Technology
3,4,Birla Institute of Technology
4,5,International Institute of Information Technology


In [14]:
df_clg['College Name'] = df_clg['College Name'].astype(str)
df_clg.head()


,CollegeID,College Name
0,1,National Institute of Technology Rourkela
1,2,Visvesvaraya National Institute of Technology ...
2,3,Netaji Subhas University of Technology
3,4,Birla Institute of Technology
4,5,International Institute of Information Technology


In [15]:
clg_txt = df_clg['College Name'].tolist()
clg_embeddings = model.encode(clg_txt, normalize_embeddings=True)


In [12]:
chroma_client = chromadb.PersistentClient(path="chroma_db_api")
collection = chroma_client.get_or_create_collection(
    name="colleges",
    metadata={"hnsw:space": "cosine"}       
)

In [17]:
# choose a batch size at or below the reported maximum
batch_size = 5461

# prepare your full lists
ids        = [str(i) for i in df_clg['CollegeID']]
metadatas = [{"cllge": c} for c in clg_txt]
documents = clg_txt
embeddings = clg_embeddings.tolist()


# helper to split a list into batches
def batchify(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i : i + size]

# iterate over batches and add them one chunk at a time
for id_batch, meta_batch, doc_batch, emb_batch in zip(
        batchify(ids, batch_size),
        batchify(metadatas, batch_size),
        batchify(documents, batch_size),
        batchify(embeddings, batch_size),
    ):
    collection.add(
        ids=id_batch,
        metadatas=meta_batch,
        documents=doc_batch,
        embeddings=emb_batch
    )


In [13]:
def lookup_college_id(query: str, k: int = 1) -> list[dict]:
    # 1) Normalize & encode
    raw = normalize(query)
    q_emb = model.encode([raw], normalize_embeddings=True)[0]

    # 2) Query Chroma
    res = collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=k,
        include=['metadatas', 'distances']
    )

    # 3) Unwrap the batch dimension (we only did one query)
    ids_list       = res['ids'][0]         # e.g. ['cid1','cid2',...]
    metas_list     = res['metadatas'][0]   # e.g. [ { 'cllge': '…', … }, … ]
    distances_list = res['distances'][0]   # e.g. [0.12, 0.33, …]

    # 4) Build your output
    matches = []
    for cid, meta, dist in zip(ids_list, metas_list, distances_list):
        matches.append({
            "college_id":   cid,
            "college_name": meta.get("college", ""),
            "distance":     dist
        })

    return matches

# Now this should work without the AttributeError:
print(lookup_college_id("Indian Institute of Technology Bombay"))
print(lookup_college_id("delhi technological university" , k=3))
print(lookup_college_id("NIT", k=3))
print(lookup_college_id("NITD", k=3))


[{'college_id': '47', 'college_name': 'indian institute of technology gandhinagar', 'distance': 0.0696224570274353}]
[{'college_id': '1893', 'college_name': 'delhi institute of technology', 'distance': 0.059738993644714355}, {'college_id': '2767', 'college_name': 'delhi global institute of technology', 'distance': 0.07531154155731201}, {'college_id': '2129', 'college_name': 'delhi technical campus', 'distance': 0.08200943470001221}]
[{'college_id': '432', 'college_name': 'niit university', 'distance': 0.47304415702819824}, {'college_id': '4043', 'college_name': 'nit polytechnic', 'distance': 0.5399657487869263}, {'college_id': '2138', 'college_name': 'niilm university', 'distance': 0.5543206930160522}]
[{'college_id': '432', 'college_name': 'niit university', 'distance': 0.5390298962593079}, {'college_id': '4043', 'college_name': 'nit polytechnic', 'distance': 0.6147104501724243}, {'college_id': '578', 'college_name': 'nitra technical campus', 'distance': 0.6175847053527832}]


In [30]:
df_comp = pd.read_csv('companies.csv')
df_comp.head()

,Domain,Company,Company ID
0,accenture.com,Accenture,1
1,adobe.com,Adobe,2
2,amazon.com,Amazon,3
3,amazon.in,Amazon,3
4,avineonindia.com,Avineon India,4


In [31]:
df_comp = (df_comp
    [['Company ID', 'Company']]                # pick only the two columns
    .drop_duplicates(subset=['Company ID', 'Company'])  # remove duplicate rows
    .reset_index(drop=True)                    # optional: re‑index from 0
)

In [34]:
cmp_txt = df_comp['Company'].tolist()
print(cmp_txt[:5])  # Display first 5 company names for verification
cmp_embeddings = model.encode(cmp_txt, normalize_embeddings=True)

['Accenture', 'Adobe', 'Amazon', 'Avineon India', 'Blue Star Infotech']


In [35]:
chroma_client = chromadb.PersistentClient(path="chroma_db_2")
comp_collection = chroma_client.get_or_create_collection(
    name="company",
    metadata={"hnsw:space": "cosine"}       
)

In [36]:
comp_collection.add(
    ids=[str(i) for i in df_comp['Company ID']], 
    embeddings=cmp_embeddings.tolist(),
    metadatas=[{"company": c} for c in cmp_txt],
    documents=cmp_txt                 
)

In [38]:
def lookup_company_id(query: str, k: int = 1) -> list[dict]:
    # 1) Normalize & encode
    raw = normalize(query)
    q_emb = model.encode([raw], normalize_embeddings=True)[0]

    # 2) Query Chroma
    res = comp_collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=k,
        include=['metadatas', 'distances']
    )

    # 3) Unwrap the batch dimension (we only did one query)
    ids_list       = res['ids'][0]         # e.g. ['cid1','cid2',...]
    metas_list     = res['metadatas'][0]   # e.g. [ { 'cllge': '…', … }, … ]
    distances_list = res['distances'][0]   # e.g. [0.12, 0.33, …]

    # 4) Build your output
    matches = []
    for cid, meta, dist in zip(ids_list, metas_list, distances_list):
        matches.append({
            "company_id":   cid,
            "company_name": meta.get("company", ""),
            "distance":     dist
        })

    return matches

# Now this should work without the AttributeError:
print(lookup_company_id("TCS"))
print(lookup_company_id("Infosys", k=3))    
print(lookup_company_id("Wipro", k=3))
print(lookup_company_id("HCL", k=3))

[{'company_id': '60', 'company_name': 'TATA Consultancy Services', 'distance': 0.4901370406150818}]
[{'company_id': '36', 'company_name': 'Infosys', 'distance': -8.344650268554688e-07}, {'company_id': '33', 'company_name': 'Ids Infotech', 'distance': 0.28184396028518677}, {'company_id': '301', 'company_name': 'SISL Infotech', 'distance': 0.30938732624053955}]
[{'company_id': '65', 'company_name': 'Wipro', 'distance': 0.0}, {'company_id': '80', 'company_name': 'Wissen Technology', 'distance': 0.5065919160842896}, {'company_id': '176', 'company_name': 'Qualtech Edge', 'distance': 0.5484176874160767}]
[{'company_id': '26', 'company_name': 'HCL Technologies', 'distance': 0.2819862365722656}, {'company_id': '230', 'company_name': 'Clarivate', 'distance': 0.5963231325149536}, {'company_id': '30', 'company_name': 'Honeywell', 'distance': 0.6119521856307983}]


In [11]:
client = chromadb.PersistentClient(path="chroma_db_api")

# 1) List all collections
print("Collections:", [c.name for c in client.list_collections()])

# 2) Pick the one you care about
col = client.get_collection("skills")  

# 3) Count how many records it has
print("Skill embeddings count:", col.count())

# 4) (Optionally) Peek at the first few documents & metadata
peek = col.get(
    ids=[],       # empty list means “ignore id filter”
    limit=5       # top 5 whatever order
)
print("Sample IDs:", peek["ids"])
print("Sample metadatas:", peek["metadatas"])
print("Sample documents:", peek["documents"])

Collections: ['skills', 'colleges', 'companies']
Skill embeddings count: 353


ValueError: Expected IDs to be a non-empty list, got 0 IDs in get.